In [19]:
#!pip install pyspark

In [20]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
from pyspark.sql.functions import approx_count_distinct,collect_list
from pyspark.sql.functions import collect_set,sum,avg,max,countDistinct,count
from pyspark.sql.functions import first, last, min, mean 
from pyspark.sql.functions import sumDistinct,col

In [21]:
#create the session

conf = SparkConf().set("spark.ui.port","4050")

#create the context
import pyspark
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

ValueError: ignored

In [22]:
df = spark.read.csv("/content/diabetic_data.csv", header=True)

df.printSchema()

root
 |-- encounter_id: string (nullable = true)
 |-- patient_nbr: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- admission_type_id: string (nullable = true)
 |-- discharge_disposition_id: string (nullable = true)
 |-- admission_source_id: string (nullable = true)
 |-- time_in_hospital: string (nullable = true)
 |-- payer_code: string (nullable = true)
 |-- medical_specialty: string (nullable = true)
 |-- num_lab_procedures: string (nullable = true)
 |-- num_procedures: string (nullable = true)
 |-- num_medications: string (nullable = true)
 |-- number_outpatient: string (nullable = true)
 |-- number_emergency: string (nullable = true)
 |-- number_inpatient: string (nullable = true)
 |-- diag_1: string (nullable = true)
 |-- diag_2: string (nullable = true)
 |-- diag_3: string (nullable = true)
 |-- number_diagnoses: string (nullable = true)
 |-- max_glu_ser

In [23]:
df.show()

+------------+-----------+---------------+------+--------+------+-----------------+------------------------+-------------------+----------------+----------+--------------------+------------------+--------------+---------------+-----------------+----------------+----------------+------+------+------+----------------+-------------+---------+---------+-----------+-----------+--------------+-----------+-------------+---------+---------+-----------+------------+-------------+--------+--------+------------+----------+-------+-----------+-------+-------------------+-------------------+------------------------+-----------------------+----------------------+------+-----------+----------+
|encounter_id|patient_nbr|           race|gender|     age|weight|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|payer_code|   medical_specialty|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|diag_1|diag_2|diag_3|number_diagn

**1. Find the number of patients which have not been readmitted in hospital.**

In [24]:
# Filter the records to include only patients who have not been readmitted
not_readmitted_df = df.filter(col('readmitted') == 'No')
not_readmitted_df.count()

0

**2. Find the number of patients which have been readmitted in less than 30 days.**

In [25]:
# Filter the records to include only patients who have been readmitted in less than 30 days
readmitted_less_df = df.filter(col('readmitted') == '<30')
readmitted_less_df.count()

6958

**3.Find the number of patients which have been readmitted after 30 days.**

In [26]:
# Filter the number of patients which have been readmitted after 30 days
readmitted_after_df = df.filter(col('readmitted') == '>30')
readmitted_after_df.count()

21663

**4. Find the range of age which have been affected by diabetes more. (take help of diabetes column).**

In [36]:
# Filter the records to include only patients with diabetes
diabetes_df = df.filter(col('diabetesMed') == 'Yes')
diabetes_df.show()

+------------+-----------+---------------+------+--------+------+-----------------+------------------------+-------------------+----------------+----------+--------------------+------------------+--------------+---------------+-----------------+----------------+----------------+------+------+------+----------------+-------------+---------+---------+-----------+-----------+--------------+-----------+-------------+---------+---------+-----------+------------+-------------+--------+--------+------------+----------+-------+-----------+-------+-------------------+-------------------+------------------------+-----------------------+----------------------+------+-----------+----------+
|encounter_id|patient_nbr|           race|gender|     age|weight|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|payer_code|   medical_specialty|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|diag_1|diag_2|diag_3|number_diagn

In [37]:
# Group the data by age and count the number of diabetes occurrences for each age
age_diabetes_count_df = diabetes_df.groupBy('age').count()
age_diabetes_count_df.show()

+--------+-----+
|     age|count|
+--------+-----+
| [70-80)|11921|
|[90-100)|  966|
| [40-50)| 4531|
| [10-20)|  464|
| [20-30)|  823|
| [30-40)| 1819|
|  [0-10)|  129|
| [80-90)| 6758|
| [50-60)| 8082|
| [60-70)|10089|
+--------+-----+



In [38]:
# Find the maximum count of diabetes occurrences
max_diabetes_count = age_diabetes_count_df.select('count').agg({"count": "max"}).first()[0]
max_diabetes_count

11921

In [39]:
# Find the range of age with the maximum count of diabetes occurrences
age_range_most_affected_df = age_diabetes_count_df.filter(col('count') == max_diabetes_count)
age_range_most_affected_df.show()

+-------+-----+
|    age|count|
+-------+-----+
|[70-80)|11921|
+-------+-----+



In [40]:
# Print the range of age with the maximum count of diabetes occurrences
age_range_most_affected = age_range_most_affected_df.select("age").collect()

if age_range_most_affected:
    age_range = f"{age_range_most_affected[0]['age']} - {age_range_most_affected[-1]['age']}"
    print("Range of age most affected by diabetes: ", age_range)
else:
    print("No data available for patients with diabetes.")

Range of age most affected by diabetes:  [70-80) - [70-80)


**5. Find the number of females which have been affected by diabetes .**

In [43]:
# Filter the records to include only females with diabetes
females_with_diabetes_df = df.filter((col('gender') == 'Female') & (col('diabetesMed') == 'Yes'))
females_with_diabetes_df.count()

24474

**6. Find the number of males which have been affected by diabetes .**

In [44]:
# Filter the records to include only males with diabetes
males_with_diabetes_df = df.filter((col('gender') == 'Male') & (col('diabetesMed') == 'Yes'))
males_with_diabetes_df.count()

21107